In [ ]:
# examples/interactive_examples.ipynb
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Extended CCM Interactive Examples\n",
    "\n",
    "This notebook demonstrates the key features of the Extended CCM package with interactive visualizations."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from extended_ccm.modules.core import EnhancedCCM\n",
    "from extended_ccm.utils.visualization import CCMVisualizer\n",
    "from ipywidgets import interact, FloatSlider\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Interactive Coupled System Generation\n",
    "\n",
    "Explore how coupling strength affects causality detection."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def interactive_coupling(coupling_strength=0.4, noise_level=0.05):\n",
    "    # Generate coupled system\n",
    "    n = 1000\n",
    "    r = 3.8\n",
    "    x = np.zeros(n)\n",
    "    y = np.zeros(n)\n",
    "    \n",
    "    x[0] = 0.4\n",
    "    y[0] = 0.2\n",
    "    \n",
    "    for t in range(1, n):\n",
    "        x[t] = r * x[t-1] * (1 - x[t-1])\n",
    "        y[t] = r * y[t-1] * (1 - y[t-1]) + coupling_strength * x[t-1]\n",
    "    \n",
    "    # Add noise\n",
    "    x += np.random.normal(0, noise_level, n)\n",
    "    y += np.random.normal(0, noise_level, n)\n",
    "    \n",
    "    # Run CCM\n",
    "    ccm = EnhancedCCM(embedding_dimension=3)\n",
    "    lib_sizes = np.arange(10, 500, 50)\n",
    "    results = ccm.run_ccm(x, y, lib_sizes)\n",
    "    \n",
    "    # Plot results\n",
    "    viz = CCMVisualizer()\n",
    "    viz.plot_convergence(\n",
    "        lib_sizes,\n",
    "        [r.correlation for r in results],\n",
    "        title=f'CCM Analysis (Coupling={coupling_strength:.2f}, Noise={noise_level:.2f})'\n",
    "    )\n",
    "    plt.show()\n",
    "\n",
    "interact(\n",
    "    interactive_coupling,\n",
    "    coupling_strength=FloatSlider(min=0, max=1, step=0.1, value=0.4),\n",
    "    noise_level=FloatSlider(min=0, max=0.2, step=0.01, value=0.05)\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Signal Decomposition Exploration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def explore_decomposition(window_length=5, polyorder=2):\n",
    "    # Generate noisy signal\n",
    "    t = np.linspace(0, 10*np.pi, 1000)\n",
    "    x = np.sin(t) + 0.3 * np.sin(2*t) + 0.1 * np.random.randn(len(t))\n",
    "    \n",
    "    # Decompose signal\n",
    "    ccm = EnhancedCCM(embedding_dimension=3)\n",
    "    signal, noise = ccm.decompose_signal(x, window_length, polyorder)\
    # Plot results
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))
    
    ax1.plot(t, x, 'b-', alpha=0.5, label='Original')
    ax1.plot(t, signal, 'r-', label='Signal')
    ax1.set_title('Signal Decomposition')
    ax1.legend()
    
    ax2.plot(t, noise, 'g-', label='Noise')
    ax2.set_title('Extracted Noise')
    ax2.legend()
    
    ax3.hist(noise, bins=50, density=True, alpha=0.7)
    ax3.set_title('Noise Distribution')
    
    plt.tight_layout()
    plt.show()

interact(
    explore_decomposition,
    window_length=FloatSlider(min=3, max=21, step=2, value=5),
    polyorder=FloatSlider(min=1, max=5, step=1, value=2)
)
"""
## 3. Multivariate Analysis with Interactive Network Visualization
"""

def interactive_network_analysis(threshold=0.05):
    # Generate multivariate data
    n = 1000
    t = np.linspace(0, 20*np.pi, n)
    
    # Create coupled oscillators
    x = np.sin(t)
    y = np.sin(t + np.pi/4) + 0.3 * x
    z = np.sin(t + np.pi/2) + 0.2 * y
    w = np.sin(t + 3*np.pi/4) + 0.15 * z
    
    data = {
        'x': x + 0.1 * np.random.randn(n),
        'y': y + 0.1 * np.random.randn(n),
        'z': z + 0.1 * np.random.randn(n),
        'w': w + 0.1 * np.random.randn(n)
    }
    
    # Run multivariate CCM
    from extended_ccm.modules.multivariate import MultivariateCCM
    mccm = MultivariateCCM(embedding_dimension=3)
    
    # Analyze all pairs
    variables = list(data.keys())
    ccm_results = {}
    lib_sizes = np.arange(50, 500, 50)
    
    for i, var1 in enumerate(variables):
        for var2 in variables[i+1:]:
            result1 = mccm.run_multivariate_ccm(data, var2, lib_sizes)
            result2 = mccm.run_multivariate_ccm(data, var1, lib_sizes)
            
            ccm_results[(var1, var2)] = (
                result1['results'][-1].correlation,
                result1['results'][-1].significance
            )
            ccm_results[(var2, var1)] = (
                result2['results'][-1].correlation,
                result2['results'][-1].significance
            )
    
    # Visualize network
    viz = CCMVisualizer()
    viz.plot_causality_network(
        ccm_results,
        significance_threshold=threshold,
        node_layout='circular'
    )
    plt.show()

interact(
    interactive_network_analysis,
    threshold=FloatSlider(min=0.01, max=0.1, step=0.01, value=0.05)
)

"""
## 4. Nonlinearity Analysis and Surrogate Testing
"""

def explore_nonlinearity(noise_level=0.1, n_surrogates=20):
    from extended_ccm.modules.nonlinear import NonlinearityTests
    
    # Generate nonlinear time series
    t = np.linspace(0, 10*np.pi, 500)
    x = np.sin(t) + 0.5 * np.sin(2*t) + noise_level * np.random.randn(len(t))
    
    # Run nonlinearity tests
    nlt = NonlinearityTests(n_surrogates=n_surrogates)
    
    # Generate surrogates and compute statistics
    surrogates = [nlt.generate_surrogate(x) for _ in range(n_surrogates)]
    orig_stat = nlt.time_reversal_asymmetry(x)
    surr_stats = [nlt.time_reversal_asymmetry(s) for s in surrogates]
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot original and example surrogate
    ax1.plot(t, x, 'b-', label='Original', alpha=0.7)
    ax1.plot(t, surrogates[0], 'r-', label='Surrogate Example', alpha=0.5)
    ax1.set_title('Time Series Comparison')
    ax1.legend()
    
    # Plot statistic distribution
    ax2.hist(surr_stats, bins=20, density=True, alpha=0.5, 
             label='Surrogate Statistics')
    ax2.axvline(orig_stat, color='r', linestyle='--', 
                label='Original Statistic')
    ax2.set_title('Nonlinearity Test Results')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print results
    p_value = np.mean(np.abs(surr_stats) >= np.abs(orig_stat))
    print(f"Nonlinearity test p-value: {p_value:.3f}")
    print(f"Is nonlinear (α=0.05): {p_value < 0.05}")

interact(
    explore_nonlinearity,
    noise_level=FloatSlider(min=0, max=0.5, step=0.05, value=0.1),
    n_surrogates=FloatSlider(min=10, max=50, step=5, value=20)
)

"""
## 5. Performance Analysis
"""

def analyze_performance(n_points=1000, n_jobs=1):
    import time
    from extended_ccm.utils.performance import CCMPerformance
    
    # Generate data
    t = np.linspace(0, 20*np.pi, n_points)
    x = np.sin(t) + 0.1 * np.random.randn(n_points)
    y = np.roll(x, 5) + 0.1 * np.random.randn(n_points)
    
    # Initialize
    ccm = EnhancedCCM(embedding_dimension=3)
    perf = CCMPerformance(n_jobs=n_jobs)
    
    lib_sizes = np.arange(100, n_points//2, 100)
    
    # Time standard computation
    start_time = time.time()
    standard_results = ccm.run_ccm(x, y, lib_sizes[:5])
    standard_time = time.time() - start_time
    
    # Time optimized computation
    start_time = time.time()
    optimized_results = perf.optimize_computation(ccm, x, y, lib_sizes[:5])
    optimized_time = time.time() - start_time
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Timing comparison
    times = [standard_time, optimized_time]
    ax1.bar(['Standard', 'Optimized'], times)
    ax1.set_title('Computation Time Comparison')
    ax1.set_ylabel('Time (seconds)')
    
    # Results comparison
    standard_corr = [r.correlation for r in standard_results]
    optimized_corr = [r.correlation for r in optimized_results]
    
    ax2.plot(lib_sizes[:5], standard_corr, 'b-', label='Standard')
    ax2.plot(lib_sizes[:5], optimized_corr, 'r--', label='Optimized')
    ax2.set_title('Results Comparison')
    ax2.set_xlabel('Library Size')
    ax2.set_ylabel('Correlation')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nPerformance Summary:")
    print(f"Standard computation time: {standard_time:.3f} seconds")
    print(f"Optimized computation time: {optimized_time:.3f} seconds")
    print(f"Speedup factor: {standard_time/optimized_time:.2f}x")

interact(
    analyze_performance,
    n_points=FloatSlider(min=500, max=5000, step=500, value=1000),
    n_jobs=FloatSlider(min=1, max=8, step=1, value=1)
)